In [ ]:
# Sistemdeki olası çakışmaları temizle ve gerekli araçları kur
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver libnss3
!pip install selenium webdriver-manager

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Sürücü yollarını sisteme tanıtıyoruz
os.environ['PATH'] += ":/usr/lib/chromium-browser/"
os.environ['PATH'] += ":/usr/lib/chromium-browser/chromedriver/"

In [ ]:
# Gerekli kütüphaneleri yükleyelim
!pip install beautifulsoup4 pandas

from bs4 import BeautifulSoup
import json
import pandas as pd

def gratis_html_cozumle(html_content):
    """
    Gratis (veya benzeri Next.js tabanlı) HTML içeriğinden ürün bilgilerini çeker.
    Öncelik sırası:
    1. JSON-LD (Google için hazırlanan yapısal veri - En garantisi)
    2. Meta Etiketleri (Sosyal medya paylaşımları için olan veri)
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    product_info = {}

    print("Analiz yapılıyor...")

    # --- YÖNTEM 1: JSON-LD Verisi (En Temiz Yöntem) ---
    scripts = soup.find_all('script', {'type': 'application/ld+json'})
    json_found = False

    for script in scripts:
        try:
            data = json.loads(script.string)

            # Bazen JSON bir liste olarak gelir, bazen tek obje
            if isinstance(data, list):
                # Liste içindeki "Product" tipindeki veriyi arayalım
                for item in data:
                    if item.get('@type') == 'Product':
                        data = item
                        break

            if data.get('@type') == 'Product':
                print("✅ JSON-LD Ürün verisi bulundu.")
                product_info['Ürün Adı'] = data.get('name')
                product_info['Marka'] = data.get('brand', {}).get('name') if isinstance(data.get('brand'), dict) else data.get('brand')
                product_info['Resim'] = data.get('image')
                product_info['SKU'] = data.get('sku')

                # Fiyat bilgisi "offers" altındadır
                offers = data.get('offers')
                if isinstance(offers, list): # Bazen liste olabilir
                    offers = offers[0]

                if isinstance(offers, dict):
                    product_info['Fiyat'] = offers.get('price')
                    product_info['Para Birimi'] = offers.get('priceCurrency')
                    product_info['Stok Durumu'] = "Stokta Var" if "InStock" in offers.get('availability', '') else "Stokta Yok"

                json_found = True
                product_info['Kaynak'] = 'JSON-LD'
                break
        except Exception as e:
            continue

    # --- YÖNTEM 2: Meta Etiketleri (Yedek Yöntem) ---
    # Eğer JSON-LD'den veri gelmediyse veya eksikse Meta taglerine bak
    if not json_found or not product_info.get('Fiyat'):
        print("⚠️ JSON verisi tam değil, Meta etiketleri taranıyor...")

        if 'Ürün Adı' not in product_info:
            og_title = soup.find('meta', property='og:title')
            product_info['Ürün Adı'] = og_title['content'] if og_title else "Bulunamadı"

        # Gratis gibi sitelerde fiyat bazen meta taglerinde açıkça yazar
        price_meta = soup.find('meta', property='product:price:amount')
        currency_meta = soup.find('meta', property='product:price:currency')

        if price_meta:
            product_info['Fiyat'] = price_meta['content']
            product_info['Kaynak'] = 'Meta-Tag'

        if currency_meta:
            product_info['Para Birimi'] = currency_meta['content']

        if 'Resim' not in product_info:
            og_image = soup.find('meta', property='og:image')
            product_info['Resim'] = og_image['content'] if og_image else None

    return product_info

# --- ÇALIŞTIRMA KISMI ---

# 1. Dosyayı Colab'a yüklediğinizi varsayıyoruz: 'responseürün.html'
dosya_adi = 'responseürün.html'

try:
    with open(dosya_adi, 'r', encoding='utf-8') as f:
        html_icerigi = f.read()

    # Fonksiyonu çalıştır
    sonuc = gratis_html_cozumle(html_icerigi)

    # Sonucu göster
    if sonuc:
        df = pd.DataFrame([sonuc])
        print("\n--- SONUÇLAR ---")
        display(df)

        # İsterseniz Excel'e de kaydedebilirsiniz
        # df.to_excel("gratis_urun_analizi.xlsx", index=False)
    else:
        print("Ürün bilgisi bulunamadı.")

except FileNotFoundError:
    print(f"Hata: '{dosya_adi}' dosyası bulunamadı. Lütfen dosyayı Colab'ın sol menüsündeki dosya bölümüne yükleyin.")
except Exception as e:
    print(f"Bir hata oluştu: {e}")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# --- AYARLAR ---
# İsterseniz tüm sayfaları gezmesi için sayıyı artırabilirsiniz (Örn: 20)
TARANACAK_SAYFA_SAYISI = 3
KATEGORI_URL = "https://www.eveshop.com.tr/collections/makyaj"

# Tarayıcı gibi görünmek için başlıklar
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

print(f"🚀 Eve Shop Veri Çekme Başlatıldı: {KATEGORI_URL}")
print("-" * 60)

tum_urunler = []

for sayfa in range(1, TARANACAK_SAYFA_SAYISI + 1):
    url = f"{KATEGORI_URL}?page={sayfa}"
    print(f"📄 Sayfa {sayfa} taranıyor...")

    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        if response.status_code != 200:
            print(f"   ⚠️ Sayfa yüklenemedi (Kod: {response.status_code})")
            continue

        soup = BeautifulSoup(response.content, "html.parser")

        # --- ÜRÜNLERİ BULMA ---
        # Sizin dosyanızdan tespit ettiğimiz 'card-product' sınıfını kullanıyoruz
        kartlar = soup.select(".card-product")

        print(f"   ✅ {len(kartlar)} ürün bulundu.")

        for kart in kartlar:
            try:
                # 1. Ürün Adı
                title_tag = kart.select_one(".x-card-title")
                if not title_tag: continue # İsmi olmayanları atla

                ad = title_tag.get_text(strip=True)
                link = "https://www.eveshop.com.tr" + title_tag.get('href', '')

                # 2. Fiyat
                price_tag = kart.select_one(".small-price.price span")
                fiyat = price_tag.get_text(strip=True) if price_tag else "Fiyat Yok"

                # 3. Resim
                img_tag = kart.select_one("img.preview-img")
                resim_url = "Resim Yok"
                if img_tag:
                    # Bazen src boştur, srcset veya data-src dolu olabilir
                    src = img_tag.get("src")
                    if src and "//" in src:
                        resim_url = "https:" + src if src.startswith("//") else src
                    elif img_tag.get("srcset"):
                        # srcset varsa ilk linki al
                        resim_url = "https:" + img_tag.get("srcset").split(",")[0].split(" ")[0].lstrip()

                tum_urunler.append({
                    "Sayfa": sayfa,
                    "Ürün Adı": ad,
                    "Fiyat": fiyat,
                    "Resim": resim_url,
                    "Link": link
                })

            except Exception as e:
                continue

        # Siteyi yormamak için kısa bekleme
        time.sleep(random.uniform(0.5, 1.0))

    except Exception as e:
        print(f"   ❌ Hata: {e}")

# --- SONUÇLARI KAYDETME ---
if tum_urunler:
    df = pd.DataFrame(tum_urunler)

    # Fiyat temizliği (TL ve gereksiz boşlukları silme)
    df["Fiyat"] = df["Fiyat"].astype(str).str.replace("TL", "").str.strip()

    print("\n" + "="*60)
    print(f"🎉 TOPLAM {len(df)} ÜRÜN BAŞARIYLA ÇEKİLDİ!")
    print("="*60)
    print(df[["Ürün Adı", "Fiyat"]].head())

    dosya_adi = "eveshop_makyaj_urunleri.csv"
    df.to_csv(dosya_adi, index=False)
    print(f"\n💾 Dosya kaydedildi: {dosya_adi}")

    # İndirme bağlantısı (Colab için)
    try:
        from google.colab import files
        files.download(dosya_adi)
    except:
        pass
else:
    print("⚠️ Hiç ürün çekilemedi. Site yapısı değişmiş olabilir.")
    print("Veri çekilemedi.")

In [ ]:
!pip install cloudscraper

In [ ]:
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import HTML

def trendyol_urun_analiz(dosya_yolu):
    """
    trendyol.txt dosyasındaki gizli JSON verisini (SEARCH_APP_INITIAL_STATE)
    okuyarak ürünleri resimleriyle birlikte çeker.
    """
    urun_listesi = []

    try:
        # 1. Dosyayı Oku
        with open(dosya_yolu, "r", encoding="utf-8") as f:
            icerik = f.read()

        print("Dosya okunuyor...")

        # 2. JSON Verisini Regex ile Yakala
        # Trendyol tüm ürün verisini bu değişkenin içine gömer
        match = re.search(r'window\.__SEARCH_APP_INITIAL_STATE__\s*=\s*(\{.*?\});', icerik, re.DOTALL)

        if match:
            json_str = match.group(1)
            data = json.loads(json_str)

            # Ürün listesi genellikle 'products' anahtarı altındadır
            products = data.get('products', [])

            print(f"✅ JSON verisi bulundu. {len(products)} ürün işleniyor...")

            for p in products:
                # Marka
                marka = p.get('brand', {}).get('name', 'Belirtilmemiş')

                # Fiyat (İndirimli fiyat öncelikli)
                fiyat_info = p.get('price', {})
                satis_fiyati = fiyat_info.get('sellingPrice')
                orijinal_fiyat = fiyat_info.get('originalPrice')

                # Resim (Genellikle ilk resim ana resimdir)
                resimler = p.get('images', [])
                resim_url = ""
                if resimler:
                    # Trendyol resim yolları bazen tam URL, bazen sadece path olabilir
                    if resimler[0].startswith('http'):
                        resim_url = resimler[0]
                    else:
                        resim_url = f"https://cdn.dsmcdn.com/{resimler[0]}"

                # Ürün Linki
                link = "https://www.trendyol.com" + p.get('url', '')

                urun_listesi.append({
                    "Görsel": f'<img src="{resim_url}" width="60">', # Tabloda göstermek için HTML
                    "Marka": marka,
                    "Ürün Adı": p.get('name'),
                    "Fiyat (TL)": satis_fiyati,
                    "Resim Linki": resim_url,
                    "Ürün Linki": link
                })
        else:
            print("⚠️ JSON verisi bulunamadı. HTML yapısı değişmiş olabilir.")
            # Yedek plan olarak BeautifulSoup ile HTML parse eklenebilir ama
            # Trendyol'da class isimleri dinamik olduğu için JSON yöntemi çok daha sağlıklıdır.

    except FileNotFoundError:
        return pd.DataFrame()
    except Exception as e:
        print(f"Hata oluştu: {e}")
        return pd.DataFrame()

    return pd.DataFrame(urun_listesi)

# --- ÇALIŞTIRMA ---

dosya_adi = "trendyol.txt" # Colab'a yüklediğiniz dosya adı
df = trendyol_urun_analiz(dosya_adi)

if not df.empty:
    # Pandas ayarları: Linklerin tamamını göster
    pd.set_option('display.max_colwidth', None)

    print("\n--- Bulunan Ürünler (İlk 10) ---")

    # HTML formatında tabloyu göster (Resimlerin görünmesi için)
    display(HTML(df.head(10).to_html(escape=False)))

    # Excel'e kaydetmek için (Resimler link olarak kaydedilir)
    # df_excel = df.drop(columns=['Görsel']) # HTML etiketini kaldır
    # df_excel.to_excel("trendyol_urunler.xlsx", index=False)
else:
    print("Ürün bulunamadı.")

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

# Dosya yolunu belirle
file_path = 'trendyol.txt'

def parse_trendyol_html(file_path):
    try:
        # Dosyayı okuma
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')

        # Ürün kartlarını bulma
        products = soup.find_all('div', class_='product-card-container')

        product_list = []

        for product in products:
            try:
                # Veri özelliklerinden (data attributes) bilgileri çekme
                content_id = product.get('data-contentid')
                merchant_id = product.get('data-merchantid')

                # Fiyat bilgilerini al
                price = product.get('data-productprice')
                discounted_price = product.get('data-productdiscountedprice')

                # Puan bilgilerini al
                rating_score = product.get('data-rateaverage')
                rating_count = product.get('data-ratecount')

                # Linki oluştur
                data_url = product.get('data-url')
                product_link = "https://www.trendyol.com" + data_url if data_url else "Link Yok"

                # Görseli çekme
                image_tag = product.find('img', class_='pc-image')
                image_url = image_tag['src'] if image_tag else "Görsel Yok"

                # Marka ve Ürün Adını HTML içerisinden çekme
                title_div = product.find('p', class_='pc-title')
                if title_div:
                    brand_span = title_div.find('span', class_='pc-title-brand-name')
                    name_span = title_div.find('span', class_='pc-title-product-name')

                    extracted_brand = brand_span.text.strip() if brand_span else "Marka Yok"
                    extracted_name = name_span.text.strip() if name_span else "Ürün Adı Yok"
                else:
                    extracted_brand = "Bilinmiyor"
                    extracted_name = "Bilinmiyor"

                # Veriyi listeye ekleme
                product_data = {
                    'Marka': extracted_brand,
                    'Ürün Adı': extracted_name,
                    'Fiyat (TL)': price,
                    'İndirimli Fiyat (TL)': discounted_price,
                    'Puan': rating_score,
                    'Değerlendirme Sayısı': rating_count,
                    'Ürün Linki': product_link
                }

                product_list.append(product_data)

            except Exception as e:
                continue

        return product_list

    except FileNotFoundError:
        return None

# Fonksiyonu çalıştır
data = parse_trendyol_html(file_path)

# Sonuçları DataFrame'e çevirip gösterme
if data:
    df = pd.DataFrame(data)

    # Fiyatları sayısal değere çevirme
    df['Fiyat (TL)'] = pd.to_numeric(df['Fiyat (TL)'], errors='coerce')
    df['İndirimli Fiyat (TL)'] = pd.to_numeric(df['İndirimli Fiyat (TL)'], errors='coerce')

    print(f"Toplam {len(df)} adet ürün bulundu.\n")

    # Hata veren display() yerine print() kullanıyoruz
    print(df.head().to_string())

    # İsterseniz Excel olarak kaydetmek için aşağıdaki satırın başındaki # işaretini kaldırın
    # df.to_excel('trendyol_kozmetik.xlsx', index=False)
else:
    print("Dosya bulunamadı veya veri çekilemedi.")

In [ ]:
# Mevcut yüklemeleri temizle ve uygun sürümleri kur
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def setup_driver():
    options = Options()
    options.add_argument('--headless') # Tarayıcıyı arkaplanda çalıştır
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(options=options)
    return driver

def get_products_from_page(driver, url):
    print(f"Listeleme sayfası taranıyor: {url}")
    driver.get(url)
    time.sleep(3) # Sayfanın yüklenmesi için bekle

    # Sayfayı aşağı kaydır (Lazy load tetiklemek için)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Trendyol listeleme yapısı genellikle 'p-card-wrppr' class'ı içindedir
    product_cards = soup.find_all('div', class_='p-card-wrppr')

    products = []
    for card in product_cards:
        try:
            # Link
            link_tag = card.find('a')
            full_link = "https://www.trendyol.com" + link_tag['href'] if link_tag else None

            # Marka ve İsim
            brand_tag = card.find('span', class_='prdct-desc-cntnr-ttl')
            name_tag = card.find('span', class_='prdct-desc-cntnr-name')

            brand = brand_tag.text.strip() if brand_tag else "Marka Yok"
            name = name_tag.text.strip() if name_tag else "İsim Yok"

            # Fiyat
            price_tag = card.find('div', class_='prc-box-dscntd')
            price = price_tag.text.strip() if price_tag else "Fiyat Yok"

            products.append({
                'Marka': brand,
                'Urun_Adi': name,
                'Fiyat': price,
                'Link': full_link
            })
        except Exception as e:
            continue

    return products

def get_product_reviews(driver, product_url):
    print(f"--> Ürün detayına gidiliyor: {product_url}")
    try:
        driver.get(product_url)
        time.sleep(2)

        # Yorumlar sekmesine gitmek yerine, Trendyol ürün detayında
        # "Değerlendirmeler" kısmını HTML'den çekmeye çalışacağız.
        # Yorumlar genellikle dinamik yüklenir, bu yüzden sayfayı kaydırmalıyız.

        driver.execute_script("window.scrollTo(0, 1000);")
        time.sleep(2)

        # Yorumları çekme (Bu class isimleri zamanla değişebilir)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Yorum containerlarını bul (rnr-com-w, comment vb. olabilir)
        # Trendyol yorum yapısı genellikle "rnr-com-w" class'ı altındadır
        reviews = soup.find_all('div', class_='rnr-com-w')

        extracted_reviews = []
        for review in reviews[:5]: # Örnek olarak ilk 5 yorumu alıyoruz (Hız için)
            text_div = review.find('div', class_='comment-text')
            comment = text_div.text.strip() if text_div else ""
            if comment:
                extracted_reviews.append(comment)

        return extracted_reviews if extracted_reviews else ["Yorum bulunamadı veya çekilemedi"]

    except Exception as e:
        return [f"Hata: {str(e)}"]

# --- ANA İŞLEM ---

driver = setup_driver()
all_data = []

# Sadece ilk 2 sayfayı çekelim (Test için).
# Tüm ürünler için bu sayıyı arttırabilirsiniz ama çok uzun sürer.
max_pages = 2
base_url = "https://www.trendyol.com/butik/liste/11/kozmetik?pi="

try:
    # 1. Ürünleri Listele
    for i in range(1, max_pages + 1):
        page_url = base_url + str(i)
        products = get_products_from_page(driver, page_url)
        print(f"Sayfa {i}'den {len(products)} ürün bulundu.")

        # 2. Her ürünün detayına girip Yorumları Çek
        # Hız testi için her sayfadan sadece ilk 3 ürünü gezelim:
        for product in products[:3]:
            if product['Link']:
                reviews = get_product_reviews(driver, product['Link'])
                product['Yorumlar'] = " | ".join(reviews)
                all_data.append(product)
            else:
                continue

finally:
    driver.quit()

# Sonuçları Kaydet
df = pd.DataFrame(all_data)
print("-" * 30)
print(f"Toplam {len(df)} ürün detaylı olarak işlendi.")
print(df.head())

# Excel'e kaydet
df.to_excel("trendyol_kozmetik_detayli.xlsx", index=False)
print("Excel dosyası oluşturuldu.")

In [ ]:
# Gerekli tarayıcı ve sürücüleri kur
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium webdriver-manager pandas bs4
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def setup_driver():
    options = Options()
    options.add_argument('--headless') # Arayüzsüz mod (Hız için)
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(options=options)
    return driver

def scrape_listing_page(driver, url):
    print(f"Sayfa taranıyor: {url}")
    driver.get(url)
    time.sleep(3) # İlk yükleme beklemesi

    # Trendyol'da ürünler aşağı indikçe yüklenir (Lazy Load).
    # Sayfanın en altına kadar kademeli olarak iniyoruz.
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # Yüklenmesi için bekle
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Sayfa kaynağını al ve parse et
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Trendyol ürün kartı kapsayıcısı
    product_cards = soup.find_all('div', class_='p-card-wrppr')

    page_data = []

    for card in product_cards:
        try:
            # 1. Ana ürün container'ını bul (Veriler genellikle buradaki data- attribute'larda saklıdır)
            # Genellikle p-card-chldrn-cntnr sınıfında veya card'ın ilk div'inde olur
            container_div = card.find('div', class_='p-card-chldrn-cntnr')

            # Eğer container yoksa direkt card'a bak
            target_elem = container_div if container_div else card

            # --- VERİLERİ ÇEK ---

            # Temel Bilgiler (Data Attribute'lardan)
            content_id = target_elem.get('data-contentid') or target_elem.get('data-id')
            brand = target_elem.get('data-brandname') # Bazen boştur, aşağıda text ile kontrol edeceğiz
            price = target_elem.get('data-productprice')

            # İstenilen Yıldız ve Yorum Sayısı (Data Attribute'lardan)
            rating_score = target_elem.get('data-rateaverage') # Örn: 4.5
            rating_count = target_elem.get('data-ratecount')   # Örn: 1250

            # Link
            a_tag = card.find('a')
            product_link = "https://www.trendyol.com" + a_tag['href'] if a_tag else "Link Yok"

            # Görsel
            img_tag = card.find('img', class_='p-card-img')
            if not img_tag:
                # Bazen class ismi farklı olabilir
                img_tag = card.find('img')
            image_url = img_tag['src'] if img_tag else "Görsel Yok"

            # Eğer data attribute'da marka yoksa text'ten al
            if not brand:
                brand_span = card.find('span', class_='prdct-desc-cntnr-ttl')
                brand = brand_span.text.strip() if brand_span else "Marka Belirtilmemiş"

            # Ürün Adı (Genelde text içindedir)
            name_span = card.find('span', class_='prdct-desc-cntnr-name')
            product_name = name_span.text.strip() if name_span else "Ürün Adı Yok"

            # Fiyat (Yedek yöntem: Text üzerinden)
            if not price:
                price_div = card.find('div', class_='prc-box-dscntd')
                price = price_div.text.strip() if price_div else "Fiyat Yok"

            # Listeye Ekle
            page_data.append({
                'Marka': brand,
                'Ürün Adı': product_name,
                'Fiyat': price,
                'Puan (Yıldız)': rating_score if rating_score else "0",
                'Yorum Sayısı': rating_count if rating_count else "0",
                'Görsel Linki': image_url,
                'Ürün Linki': product_link
            })

        except Exception as e:
            continue

    return page_data

# --- ANA İŞLEM ---

driver = setup_driver()
all_results = []

# Sayfa sayısı (Tüm ürünler için arttırabilirsiniz)
# Kozmetik kategorisinde çok fazla ürün olduğu için 3 sayfa örnek olarak ayarlandı.
total_pages_to_scrape = 3
base_url = "https://www.trendyol.com/butik/liste/11/kozmetik?pi="

try:
    for i in range(1, total_pages_to_scrape + 1):
        target_url = base_url + str(i)
        products = scrape_listing_page(driver, target_url)
        print(f"Sayfa {i} tamamlandı. {len(products)} ürün çekildi.")
        all_results.extend(products)

finally:
    driver.quit()

# Veriyi Kaydetme ve Gösterme
df = pd.DataFrame(all_results)

# Veriyi temizleme (Sayısal dönüşümler)
df['Puan (Yıldız)'] = pd.to_numeric(df['Puan (Yıldız)'], errors='coerce').fillna(0)
df['Yorum Sayısı'] = pd.to_numeric(df['Yorum Sayısı'], errors='coerce').fillna(0)

print("-" * 50)
print(f"Toplam {len(df)} ürün bulundu.")
print("-" * 50)

# İlk 5 satırı göster
print(df.head().to_string())

# Excel'e kaydet
df.to_excel("trendyol_kozmetik_ozet.xlsx", index=False)
print("\nVeriler 'trendyol_kozmetik_ozet.xlsx' dosyasına kaydedildi.")

In [ ]:
import requests
import json
import re
import pandas as pd
import time
from IPython.display import HTML

def trendyol_coklu_sayfa_getir(kategori_url, sayfa_sayisi=3):
    """
    Trendyol'da sonsuz kaydırma mantığını simüle ederek birden fazla sayfadan ürün çeker.
    kategori_url: Ürünlerin çekileceği link (Örn: https://www.trendyol.com/kadin-elbise-x-g1-c56)
    sayfa_sayisi: Kaç sayfa (scroll) ilerleneceği.
    """

    tum_urunler = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    print(f"Toplam {sayfa_sayisi} sayfa taranacak...\n")

    for sayfa in range(1, sayfa_sayisi + 1):
        # URL'yi sayfa parametresine göre düzenle
        # Eğer URL'de zaten ? varsa &pi= ekle, yoksa ?pi= ekle
        if "?" in kategori_url:
            hedef_url = f"{kategori_url}&pi={sayfa}"
        else:
            hedef_url = f"{kategori_url}?pi={sayfa}"

        print(f"Sayfa {sayfa} işleniyor: {hedef_url}")

        try:
            response = requests.get(hedef_url, headers=headers)
            if response.status_code != 200:
                print(f"⚠️ Sayfa {sayfa} yüklenemedi. Kod: {response.status_code}")
                continue

            # HTML içinden JSON verisini regex ile söküp alıyoruz
            match = re.search(r'window\.__SEARCH_APP_INITIAL_STATE__\s*=\s*(\{.*?\});', response.text, re.DOTALL)

            if match:
                data = json.loads(match.group(1))
                products = data.get('products', [])

                if not products:
                    print(f"⚠️ Sayfa {sayfa}'da ürün bulunamadı, döngü sonlandırılıyor.")
                    break

                for p in products:
                    # Resim URL'sini düzelt
                    img_url = p.get('images', [''])[0]
                    if img_url and not img_url.startswith('http'):
                        img_url = f"https://cdn.dsmcdn.com/{img_url}"

                    item = {
                        "Görsel": f'<img src="{img_url}" width="60">',
                        "Ürün Adı": p.get('name'),
                        "Marka": p.get('brand', {}).get('name'),
                        "Fiyat": p.get('price', {}).get('sellingPrice'),
                        "Değerlendirme": p.get('ratingScore', {}).get('averageRating'),
                        "Yorum Sayısı": p.get('ratingScore', {}).get('totalCount'),
                        "Link": "https://www.trendyol.com" + p.get('url'),
                        "Sayfa": sayfa # Hangi sayfadan geldiği
                    }
                    tum_urunler.append(item)
            else:
                print(f"⚠️ Sayfa {sayfa}'da veri bloğu bulunamadı.")

            # Trendyol'u yormamak için her sayfa arası biraz bekleyelim
            time.sleep(1.5)

        except Exception as e:
            print(f"Hata oluştu: {e}")
            continue

    return pd.DataFrame(tum_urunler)

# --- KULLANIM ---

# Buraya çekmek istediğiniz kategori linkini yapıştırın
# Örnek: Kozmetik kategorisi
link = "https://www.trendyol.com/kozmetik-x-c89"

# Kaç sayfa aşağı inilsin? (Örn: 5 sayfa yaklaşık 120 ürün eder)
df = trendyol_coklu_sayfa_getir(link, sayfa_sayisi=3)

# Sonuçları göster
if not df.empty:
    print(f"\n✅ Toplam {len(df)} ürün başarıyla çekildi.")

    # Pandas ayarları (Linklerin tamamını görmek için)
    pd.set_option('display.max_colwidth', None)

    # Tabloyu resimlerle beraber göster
    display(HTML(df.head(10).to_html(escape=False)))

    # Excel'e kaydet (Resim HTML etiketini kaldırarak kaydeder)
    # df_save = df.drop(columns=['Görsel'])
    # df_save.to_excel("trendyol_kozmetik_tum_sayfalar.xlsx", index=False)
    # print("Excel dosyası kaydedildi.")
else:
    print("Hiç ürün çekilemedi.")

In [ ]:
# Önce gerekli ve güçlü kütüphaneyi yükleyelim
!pip install cloudscraper beautifulsoup4 pandas

import cloudscraper
import json
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from IPython.display import HTML

def trendyol_guclu_getir(kategori_url, sayfa_sayisi=3):
    """
    Trendyol bot korumasını aşmaya çalışarak verileri çeker.
    Hem JSON verisini hem de HTML yapısını tarar.
    """
    # Cloudscraper, tarayıcı taklidi yapar
    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
    )

    tum_urunler = []

    print(f"--- Tarama Başlıyor: {sayfa_sayisi} Sayfa ---\n")

    for sayfa in range(1, sayfa_sayisi + 1):
        # Sayfa parametresini ayarla
        if "?" in kategori_url:
            hedef_url = f"{kategori_url}&pi={sayfa}"
        else:
            hedef_url = f"{kategori_url}?pi={sayfa}"

        print(f"🔍 Sayfa {sayfa} taranıyor...")

        try:
            # Standart requests yerine scraper kullanıyoruz
            response = scraper.get(hedef_url)

            if response.status_code != 200:
                print(f"❌ Erişim engellendi! Durum Kodu: {response.status_code}")
                continue

            # İçeriği analiz et
            soup = BeautifulSoup(response.text, 'html.parser')
            sayfa_urunleri = []

            # --- PLAN A: Gizli JSON Verisi (En Temiz Yöntem) ---
            match = re.search(r'window\.__SEARCH_APP_INITIAL_STATE__\s*=\s*(\{.*?\});', response.text, re.DOTALL)
            json_basarili = False

            if match:
                try:
                    data = json.loads(match.group(1))
                    products = data.get('products', [])
                    if products:
                        for p in products:
                            # Fiyatı bul
                            fiyat = p.get('price', {}).get('discountedPrice') or p.get('price', {}).get('sellingPrice')

                            # Resim
                            resim = p.get('images', [''])[0]
                            if resim and not resim.startswith('http'): resim = f"https://cdn.dsmcdn.com/{resim}"

                            sayfa_urunleri.append({
                                "Yöntem": "JSON",
                                "Görsel": f'<img src="{resim}" width="50">',
                                "Marka": p.get('brand', {}).get('name'),
                                "Ürün Adı": p.get('name'),
                                "Fiyat": fiyat,
                                "Link": "https://www.trendyol.com" + p.get('url'),
                                "Sayfa": sayfa
                            })
                        json_basarili = True
                        print(f"   ✅ JSON Yöntemiyle {len(products)} ürün çekildi.")
                except:
                    pass # JSON hatası olursa HTML'e geç

            # --- PLAN B: HTML Kartları (Yedek Yöntem) ---
            # Eğer JSON boş geldiyse veya bulunamadıysa HTML taglerini tara
            if not json_basarili:
                print("   ⚠️ JSON verisi korumalı, HTML kartları taranıyor...")
                kartlar = soup.find_all('div', class_='p-card-chldrn-cntnr') # Ürün kartı kapsayıcısı

                # Alternatif kart yapısı kontrolü
                if not kartlar:
                    kartlar = soup.select('.product-card')

                for kart in kartlar:
                    try:
                        # Marka ve İsim
                        marka_tag = kart.select_one('.product-brand') or kart.select_one('.brand')
                        isim_tag = kart.select_one('.product-name') or kart.select_one('.name')
                        marka = marka_tag.text.strip() if marka_tag else "Bilinmiyor"
                        isim = isim_tag.text.strip() if isim_tag else ""

                        # Link
                        link_tag = kart.find('a') if kart.name != 'a' else kart
                        link = "https://www.trendyol.com" + link_tag['href'] if link_tag else None

                        # Fiyat (Çeşitli class isimleri olabilir)
                        fiyat_tag = kart.select_one('.price-value') or kart.select_one('.prc-box-dscntd') or kart.select_one('.prc-box-sllng')
                        fiyat = fiyat_tag.text.strip() if fiyat_tag else "Tükendi/Görünmüyor"

                        # Resim
                        img_tag = kart.find('img')
                        resim = img_tag['src'] if img_tag else ""

                        if isim: # Boş kart değilse ekle
                            sayfa_urunleri.append({
                                "Yöntem": "HTML",
                                "Görsel": f'<img src="{resim}" width="50">',
                                "Marka": marka,
                                "Ürün Adı": isim,
                                "Fiyat": fiyat,
                                "Link": link,
                                "Sayfa": sayfa
                            })
                    except:
                        continue

                if sayfa_urunleri:
                    print(f"   ✅ HTML Yöntemiyle {len(sayfa_urunleri)} ürün kurtarıldı.")
                else:
                    print("   ❌ Bu sayfadan veri alınamadı.")

            tum_urunler.extend(sayfa_urunleri)

            # Sunucuyu şüphelendirmemek için bekleme süresi (rastgele)
            time.sleep(2)

        except Exception as e:
            print(f"   ❌ Hata: {e}")
            continue

    return pd.DataFrame(tum_urunler)

# --- ÇALIŞTIRMA ---
link = "https://www.trendyol.com/kozmetik-x-c89"

# İlk 2 sayfayı deneyelim (Çok fazla sayfa denemek engellenme riskini artırır)
df_sonuc = trendyol_guclu_getir(link, sayfa_sayisi=2)

if not df_sonuc.empty:
    print(f"\n🎉 TOPLAM {len(df_sonuc)} ÜRÜN BULUNDU\n")
    # Link sütununu tam göstermek için ayar
    pd.set_option('display.max_colwidth', None)
    display(HTML(df_sonuc.head(10).to_html(escape=False)))
else:
    print("\n⚠️ Veri çekilemedi. Trendyol IP adresinizi geçici olarak engellemiş olabilir.")
    print("Çözüm: VPN değiştirip tekrar deneyin veya 'Sayfa 1' linkini tarayıcıda açıp CAPTCHA varsa çözün.")

In [ ]:
# 1. GEREKLİ KÜTÜPHANELERİ YÜKLE
!pip install cloudscraper beautifulsoup4 pandas

# 2. İMPORTLAR
import cloudscraper
import json
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from IPython.display import display, HTML

# 3. VERİ ÇEKME FONKSİYONU
def trendyol_profesyonel_kaziyici(kategori_url, sayfa_sayisi=3):
    """
    Trendyol'dan CloudScraper ile bot korumasını aşarak veri çeker.
    Hem JSON hem HTML yöntemlerini dener.
    """
    # Cloudscraper ile gerçek bir tarayıcı gibi davranıyoruz
    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
    )

    tum_urunler = []
    print(f"🚀 Tarama Başlatılıyor: {kategori_url}")
    print(f"📄 Toplam {sayfa_sayisi} sayfa taranacak...\n")

    for sayfa in range(1, sayfa_sayisi + 1):
        # Sayfa parametresini URL'ye ekle
        separator = "&" if "?" in kategori_url else "?"
        hedef_url = f"{kategori_url}{separator}pi={sayfa}"

        print(f"🔄 Sayfa {sayfa} işleniyor...", end=" ")

        try:
            # İsteği gönder
            response = scraper.get(hedef_url)

            if response.status_code != 200:
                print(f"❌ Erişim Hatası (Kod: {response.status_code})")
                continue

            # İçeriği parse et
            soup = BeautifulSoup(response.text, 'html.parser')
            sayfa_verisi = []
            json_bulundu = False

            # --- YÖNTEM 1: GİZLİ JSON VERİSİ (EN TEMİZ) ---
            # Trendyol verileri bu JavaScript değişkeninde tutar
            script_pattern = re.compile(r'window\.__SEARCH_APP_INITIAL_STATE__\s*=\s*(\{.*?\});', re.DOTALL)
            script_tag = soup.find('script', text=script_pattern)

            if script_tag:
                match = script_pattern.search(script_tag.string)
                if match:
                    try:
                        data = json.loads(match.group(1))
                        products = data.get('products', [])

                        for p in products:
                            # Verileri güvenli şekilde al
                            fiyat = p.get('price', {}).get('discountedPrice') or p.get('price', {}).get('sellingPrice')
                            resim_url = p.get('images', [''])[0]
                            if resim_url and not resim_url.startswith('http'):
                                resim_url = f"https://cdn.dsmcdn.com/{resim_url}"

                            sayfa_verisi.append({
                                "Sayfa": sayfa,
                                "Marka": p.get('brand', {}).get('name'),
                                "Ürün Adı": p.get('name'),
                                "Fiyat (TL)": fiyat,
                                "Değerlendirme": p.get('ratingScore', {}).get('averageRating'),
                                "Yorum Sayısı": p.get('ratingScore', {}).get('totalCount'),
                                "Resim": f'<img src="{resim_url}" style="width:60px; height:auto;">',
                                "Link": "https://www.trendyol.com" + p.get('url')
                            })
                        json_bulundu = True
                        print(f"✅ (JSON) {len(sayfa_verisi)} ürün çekildi.")
                    except Exception as json_err:
                        print(f"⚠️ JSON hatası: {json_err}")

            # --- YÖNTEM 2: HTML KARTLARI (YEDEK) ---
            # JSON çalışmazsa HTML elementlerini okumayı dene
            if not json_bulundu or not sayfa_verisi:
                print("⚠️ JSON bulunamadı, HTML taranıyor...", end=" ")
                kartlar = soup.select('.product-card') or soup.select('.p-card-wrppr')

                for kart in kartlar:
                    try:
                        marka = kart.select_one('.brand').get_text(strip=True) if kart.select_one('.brand') else "-"
                        isim = kart.select_one('.name').get_text(strip=True) if kart.select_one('.name') else "-"

                        fiyat_tag = kart.select_one('.prc-box-dscntd') or kart.select_one('.prc-box-sllng')
                        fiyat = fiyat_tag.get_text(strip=True) if fiyat_tag else "-"

                        img_tag = kart.select_one('img')
                        resim_src = img_tag.get('src') if img_tag else ""

                        link_tag = kart.find('a')
                        link = "https://www.trendyol.com" + link_tag['href'] if link_tag else "-"

                        sayfa_verisi.append({
                            "Sayfa": sayfa,
                            "Marka": marka,
                            "Ürün Adı": isim,
                            "Fiyat (TL)": fiyat,
                            "Değerlendirme": "-",
                            "Yorum Sayısı": "-",
                            "Resim": f'<img src="{resim_src}" style="width:60px; height:auto;">',
                            "Link": link
                        })
                    except:
                        continue
                print(f"✅ (HTML) {len(sayfa_verisi)} ürün çekildi.")

            tum_urunler.extend(sayfa_verisi)

            # Sunucuyu yormamak için bekleme
            time.sleep(2)

        except Exception as e:
            print(f"❌ Beklenmedik Hata: {e}")
            continue

    return pd.DataFrame(tum_urunler)

# 4. KODU ÇALIŞTIRMA VE SONUCU GÖSTERME
# Kategori linkini buraya girin (Örn: Makyaj kategorisi)
hedef_link = "https://www.trendyol.com/kozmetik-x-c89"

# Kaç sayfa gezileceğini belirleyin (Örn: 2 sayfa)
df_sonuc = trendyol_profesyonel_kaziyici(hedef_link, sayfa_sayisi=2)

if not df_sonuc.empty:
    print(f"\n🎉 TEBRİKLER! Toplam {len(df_sonuc)} ürün başarıyla çekildi.\n")

    # Linklerin tam görünmesi için ayar
    pd.set_option('display.max_colwidth', None)

    # Tabloyu resimlerle birlikte göster
    display(HTML(df_sonuc.head(10).to_html(escape=False)))

    # Excel'e kaydetmek isterseniz aşağıdaki satırın başındaki # işaretini kaldırın
    # df_sonuc.drop(columns=['Resim']).to_excel("trendyol_urunler.xlsx", index=False)
else:
    print("\n⚠️ Veri çekilemedi. Lütfen bağlantınızı kontrol edin veya daha sonra tekrar deneyin.")

In [ ]:
# 1. GEREKLİ KÜTÜPHANELERİ YÜKLE
!pip install cloudscraper beautifulsoup4 pandas

# 2. İMPORTLAR
import cloudscraper
import json
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from IPython.display import display, HTML

# 3. VERİ ÇEKME FONKSİYONU
def trendyol_profesyonel_kaziyici(kategori_url, sayfa_sayisi=3):
    """
    Trendyol'dan CloudScraper ile bot korumasını aşarak veri çeker.
    Hem JSON hem HTML yöntemlerini dener.
    """
    # Cloudscraper ile gerçek bir tarayıcı gibi davranıyoruz
    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
    )

    tum_urunler = []
    print(f"🚀 Tarama Başlatılıyor: {kategori_url}")
    print(f"📄 Toplam {sayfa_sayisi} sayfa taranacak...\n")

    for sayfa in range(1, sayfa_sayisi + 1):
        # Sayfa parametresini URL'ye ekle
        separator = "&" if "?" in kategori_url else "?"
        hedef_url = f"{kategori_url}{separator}pi={sayfa}"

        print(f"🔄 Sayfa {sayfa} işleniyor...", end=" ")

        try:
            # İsteği gönder
            response = scraper.get(hedef_url)

            if response.status_code != 200:
                print(f"❌ Erişim Hatası (Kod: {response.status_code})")
                continue

            # İçeriği parse et
            soup = BeautifulSoup(response.text, 'html.parser')
            sayfa_verisi = []
            json_bulundu = False

            # --- YÖNTEM 1: GİZLİ JSON VERİSİ (EN TEMİZ) ---
            # Trendyol verileri bu JavaScript değişkeninde tutar
            # Regex ile script içeriğini arıyoruz
            script_content = None
            scripts = soup.find_all('script')
            for script in scripts:
                if script.string and 'window.__SEARCH_APP_INITIAL_STATE__' in script.string:
                    script_content = script.string
                    break

            if script_content:
                match = re.search(r'window\.__SEARCH_APP_INITIAL_STATE__\s*=\s*(\{.*?\});', script_content, re.DOTALL)
                if match:
                    try:
                        data = json.loads(match.group(1))
                        products = data.get('products', [])

                        for p in products:
                            # Verileri güvenli şekilde al
                            fiyat = p.get('price', {}).get('discountedPrice') or p.get('price', {}).get('sellingPrice')
                            resim_url = p.get('images', [''])[0]
                            if resim_url and not resim_url.startswith('http'):
                                resim_url = f"https://cdn.dsmcdn.com/{resim_url}"

                            sayfa_verisi.append({
                                "Sayfa": sayfa,
                                "Marka": p.get('brand', {}).get('name'),
                                "Ürün Adı": p.get('name'),
                                "Fiyat (TL)": fiyat,
                                "Değerlendirme": p.get('ratingScore', {}).get('averageRating'),
                                "Yorum Sayısı": p.get('ratingScore', {}).get('totalCount'),
                                "Resim": f'<img src="{resim_url}" style="width:60px; height:auto;">',
                                "Link": "https://www.trendyol.com" + p.get('url')
                            })
                        json_bulundu = True
                        print(f"✅ (JSON) {len(sayfa_verisi)} ürün çekildi.")
                    except Exception as json_err:
                        print(f"⚠️ JSON hatası: {json_err}")

            # --- YÖNTEM 2: HTML KARTLARI (YEDEK) ---
            # JSON çalışmazsa HTML elementlerini okumayı dene
            if not json_bulundu or not sayfa_verisi:
                print("⚠️ JSON bulunamadı, HTML taranıyor...", end=" ")
                kartlar = soup.select('.product-card') or soup.select('.p-card-wrppr')

                for kart in kartlar:
                    try:
                        marka = kart.select_one('.brand').get_text(strip=True) if kart.select_one('.brand') else "-"
                        isim = kart.select_one('.name').get_text(strip=True) if kart.select_one('.name') else "-"

                        fiyat_tag = kart.select_one('.prc-box-dscntd') or kart.select_one('.prc-box-sllng')
                        fiyat = fiyat_tag.get_text(strip=True) if fiyat_tag else "-"

                        img_tag = kart.select_one('img')
                        resim_src = img_tag.get('src') if img_tag else ""

                        link_tag = kart.find('a')
                        link = "https://www.trendyol.com" + link_tag['href'] if link_tag else "-"

                        sayfa_verisi.append({
                            "Sayfa": sayfa,
                            "Marka": marka,
                            "Ürün Adı": isim,
                            "Fiyat (TL)": fiyat,
                            "Değerlendirme": "-",
                            "Yorum Sayısı": "-",
                            "Resim": f'<img src="{resim_src}" style="width:60px; height:auto;">',
                            "Link": link
                        })
                    except:
                        continue
                print(f"✅ (HTML) {len(sayfa_verisi)} ürün çekildi.")

            tum_urunler.extend(sayfa_verisi)

            # Sunucuyu yormamak için bekleme
            time.sleep(2)

        except Exception as e:
            print(f"❌ Beklenmedik Hata: {e}")
            continue

    return pd.DataFrame(tum_urunler)

# 4. KODU ÇALIŞTIRMA VE SONUCU GÖSTERME
# Kategori linkini buraya girin (Örn: Makyaj kategorisi)
hedef_link = "https://www.trendyol.com/kozmetik-x-c89"

# Kaç sayfa gezileceğini belirleyin (Örn: 2 sayfa)
df_sonuc = trendyol_profesyonel_kaziyici(hedef_link, sayfa_sayisi=2)

if not df_sonuc.empty:
    print(f"\n🎉 TEBRİKLER! Toplam {len(df_sonuc)} ürün başarıyla çekildi.\n")

    # Linklerin tam görünmesi için ayar
    pd.set_option('display.max_colwidth', None)

    # Tabloyu resimlerle birlikte göster
    display(HTML(df_sonuc.head(10).to_html(escape=False)))

    # Excel'e kaydetmek isterseniz aşağıdaki satırın başındaki # işaretini kaldırın
    # df_sonuc.drop(columns=['Resim']).to_excel("trendyol_urunler.xlsx", index=False)
else:
    print("\n⚠️ Veri çekilemedi. Lütfen bağlantınızı kontrol edin veya daha sonra tekrar deneyin.")

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

def trendyol_html_duzeltilmis_analiz(dosya_yolu):
    """
    trendyol2.txt dosyasındaki 'product-card' yapısına özel
    HTML ayrıştırıcı (Parser). JSON yerine doğrudan HTML sınıflarını okur.
    """
    urunler = []

    try:
        with open(dosya_yolu, "r", encoding="utf-8") as f:
            html_content = f.read()

        soup = BeautifulSoup(html_content, 'html.parser')

        # Trendyol2.txt dosyasındaki doğru kart yapısı: 'product-card'
        kartlar = soup.select('.product-card')
        print(f"Toplam {len(kartlar)} adet ürün kartı bulundu.")

        for kart in kartlar:
            try:
                # 1. Marka
                marka_tag = kart.select_one('.product-brand')
                marka = marka_tag.get_text(strip=True) if marka_tag else "Marka Yok"

                # 2. Ürün Adı
                isim_tag = kart.select_one('.product-name')
                isim = isim_tag.get_text(strip=True) if isim_tag else "İsim Yok"

                # 3. Fiyat (Dosyada .price-section içinde)
                fiyat_tag = kart.select_one('.price-section')
                if not fiyat_tag:
                    # Alternatif fiyat alanları (İndirimli/Normal)
                    fiyat_tag = kart.select_one('.prc-box-dscntd') or kart.select_one('.prc-box-sllng')
                fiyat = fiyat_tag.get_text(strip=True) if fiyat_tag else "Fiyat Yok"

                # 4. Resim
                img_tag = kart.select_one('.image')
                if img_tag:
                    resim_url = img_tag.get('src') or img_tag.get('data-src')
                    # Resim linki bazen eksik gelebilir, tamamla
                    if resim_url and not resim_url.startswith('http'):
                        resim_url = f"https://cdn.dsmcdn.com{resim_url}"
                else:
                    resim_url = ""

                # 5. Link
                link = "https://www.trendyol.com" + kart.get('href') if kart.get('href') else "#"

                urunler.append({
                    "Görsel": f'<img src="{resim_url}" style="width:60px; height:auto;">',
                    "Marka": marka,
                    "Ürün Adı": isim,
                    "Fiyat": fiyat,
                    "Link": link
                })
            except Exception as e:
                continue

    except FileNotFoundError:
        return "Dosya bulunamadı."

    return pd.DataFrame(urunler)

# --- ÇALIŞTIRMA ---
# Dosya adını buraya yazın (Colab'a yüklediğiniz dosya)
dosya_adi = "trendyol2.txt"

df_sonuc = trendyol_html_duzeltilmis_analiz(dosya_adi)

if isinstance(df_sonuc, pd.DataFrame) and not df_sonuc.empty:
    print(f"\n✅ {len(df_sonuc)} ürün başarıyla ayrıştırıldı.\n")

    # Tablo ayarları
    pd.set_option('display.max_colwidth', None)
    display(HTML(df_sonuc.head(10).to_html(escape=False)))

    # Excel'e kaydet
    # df_sonuc.drop(columns=['Görsel']).to_excel("trendyol_duzeltilmis.xlsx", index=False)
else:
    print("Ürün bulunamadı. Dosya içeriğini kontrol edin.")

In [ ]:
# Gerekli güçlü ayrıştırıcıyı yükleyelim
!pip install beautifulsoup4 lxml pandas

import os
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display, HTML

def kesin_urun_cek(dosya_yolu):
    print(f"📂 Dosya kontrol ediliyor: {dosya_yolu}")

    # 1. Dosya var mı kontrol et
    if not os.path.exists(dosya_yolu):
        print(f"❌ HATA: '{dosya_yolu}' dosyası bulunamadı!")
        print("👉 Lütfen dosyayı Colab'ın sol tarafındaki dosya bölümüne yüklediğinizden emin olun.")
        return None

    # 2. Dosyayı oku
    try:
        with open(dosya_yolu, "r", encoding="utf-8") as f:
            html_icerik = f.read()
        print(f"✅ Dosya okundu. Boyut: {len(html_icerik)} karakter.")
    except Exception as e:
        print(f"❌ Dosya okunurken hata oluştu: {e}")
        return None

    # 3. HTML Ayrıştırma (lxml kullanarak daha güçlü tarama)
    print("🔍 HTML analiz ediliyor...")
    soup = BeautifulSoup(html_icerik, "lxml") # lxml, html.parser'dan daha yeteneklidir

    # Hedef sınıfı arayalım (trendyol2.txt analizine göre: 'product-card')
    kartlar = soup.select(".product-card")

    if not kartlar:
        print("⚠️ '.product-card' bulunamadı. Alternatif aranıyor...")
        kartlar = soup.select("a[href*='/p-']") # Linkinde ürün deseni olanları ara

    print(f"🎯 Tespit edilen ürün kartı sayısı: {len(kartlar)}")

    if len(kartlar) == 0:
        print("❌ Hiçbir ürün kartı tespit edilemedi. Dosya içeriği HTML olmayabilir veya bozuk olabilir.")
        return None

    # 4. Verileri Çek
    veriler = []
    basarili = 0

    for kart in kartlar:
        try:
            # Marka
            marka_tag = kart.select_one(".product-brand") or kart.select_one(".brand")
            marka = marka_tag.get_text(strip=True) if marka_tag else "Marka Yok"

            # Ürün Adı
            isim_tag = kart.select_one(".product-name") or kart.select_one(".name")
            isim = isim_tag.get_text(strip=True) if isim_tag else "İsim Yok"

            # Fiyat
            fiyat_tag = kart.select_one(".price-section") or kart.select_one(".prc-box-dscntd")
            fiyat = fiyat_tag.get_text(strip=True) if fiyat_tag else "Fiyat Yok"

            # Resim
            img_tag = kart.select_one(".image")
            img_url = ""
            if img_tag:
                img_url = img_tag.get("src") or img_tag.get("data-src")
                if img_url and not img_url.startswith("http"):
                    img_url = "https://cdn.dsmcdn.com" + img_url

            # Link
            link = "https://www.trendyol.com" + kart.get("href") if kart.get("href") else "#"

            veriler.append({
                "Görsel": f'<img src="{img_url}" style="width:60px;">' if img_url else "-",
                "Marka": marka,
                "Ürün": isim,
                "Fiyat": fiyat,
                "Link": link
            })
            basarili += 1
        except Exception as e:
            continue

    print(f"✅ Toplam {basarili} ürün başarıyla işlendi.")
    return pd.DataFrame(veriler)

# --- ÇALIŞTIR ---
dosya_adi = "trendyol2.txt" # Burası yüklediğiniz dosya adıyla aynı olmalı
df = kesin_urun_cek(dosya_adi)

if df is not None and not df.empty:
    print("\n--- SONUÇLAR (İlk 10) ---")
    pd.set_option('display.max_colwidth', None)
    display(HTML(df.head(10).to_html(escape=False)))

    # Excel'e kaydetmek isterseniz:
    # df.drop(columns=['Görsel']).to_excel("trendyol_sonuc.xlsx", index=False)

In [ ]:
# 1. GEREKLİ KÜTÜPHANELER
!pip install cloudscraper beautifulsoup4 pandas lxml

import cloudscraper
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
from IPython.display import display, HTML

def trendyol_canli_takip(kategori_url, sayfa_sayisi=3):
    """
    Doğrulanmış HTML yapısını kullanarak canlı siteden veri çeker.
    """
    # Gerçek bir tarayıcı gibi davranan kazıyıcı
    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
    )

    tum_urunler = []
    print(f"🚀 Canlı Tarama Başlıyor: {kategori_url}")
    print(f"📄 Hedef: {sayfa_sayisi} sayfa taranacak...\n")

    for sayfa in range(1, sayfa_sayisi + 1):
        # Sayfa parametresini URL'ye ekle
        separator = "&" if "?" in kategori_url else "?"
        hedef_url = f"{kategori_url}{separator}pi={sayfa}"

        print(f"🔄 Sayfa {sayfa} indiriliyor...", end=" ")

        try:
            # 1. İsteği Gönder
            response = scraper.get(hedef_url)

            if response.status_code != 200:
                print(f"❌ Erişim Engellendi (Kod: {response.status_code})")
                continue

            # 2. HTML Analizi (Doğruladığımız Yöntem)
            soup = BeautifulSoup(response.text, 'lxml')
            kartlar = soup.select(".product-card")

            if not kartlar:
                # Yedek arama (bazen yapı değişebilir)
                kartlar = soup.select(".p-card-wrppr")

            sayfa_verisi = []

            for kart in kartlar:
                try:
                    # Marka
                    marka_tag = kart.select_one(".product-brand") or kart.select_one(".brand")
                    marka = marka_tag.get_text(strip=True) if marka_tag else "-"

                    # Ürün Adı
                    isim_tag = kart.select_one(".product-name") or kart.select_one(".name")
                    isim = isim_tag.get_text(strip=True) if isim_tag else "-"

                    # Fiyat
                    fiyat_tag = kart.select_one(".price-section") or kart.select_one(".prc-box-dscntd")
                    fiyat = fiyat_tag.get_text(strip=True) if fiyat_tag else "-"

                    # Resim
                    img_tag = kart.select_one(".image") or kart.select_one("img")
                    img_url = ""
                    if img_tag:
                        img_url = img_tag.get("src") or img_tag.get("data-src")
                        if img_url and not img_url.startswith("http"):
                            img_url = "https://cdn.dsmcdn.com" + img_url

                    # Link
                    link = "https://www.trendyol.com" + kart.get("href") if kart.get("href") else "#"

                    # Listeye Ekle
                    sayfa_verisi.append({
                        "Görsel": f'<img src="{img_url}" style="width:50px;">',
                        "Marka": marka,
                        "Ürün Adı": isim,
                        "Fiyat": fiyat,
                        "Sayfa": sayfa,
                        "Link": link
                    })
                except:
                    continue

            if sayfa_verisi:
                print(f"✅ {len(sayfa_verisi)} ürün alındı.")
                tum_urunler.extend(sayfa_verisi)
            else:
                print("⚠️ Ürün bulunamadı (Bot koruması veya sayfa sonu).")

            # 3. Bekleme (Bot korumasına yakalanmamak için rastgele bekle)
            time.sleep(random.uniform(2, 4))

        except Exception as e:
            print(f"❌ Hata: {e}")
            continue

    return pd.DataFrame(tum_urunler)

# --- ÇALIŞTIRMA AYARLARI ---

# 1. Linki Yapıştırın (Örn: Cilt Bakım kategorisi)
link = "https://www.trendyol.com/cilt-bakim-x-c85"

# 2. Fonksiyonu Başlatın
df_final = trendyol_canli_takip(link, sayfa_sayisi=2)

# 3. Sonuçları Göster
if not df_final.empty:
    print(f"\n🎉 TEBRİKLER! Toplam {len(df_final)} ürün çekildi.\n")
    pd.set_option('display.max_colwidth', None)
    display(HTML(df_final.head(10).to_html(escape=False)))

    # Excel'e Kaydetmek İçin (Yorum satırını kaldırın)
    # df_final.drop(columns=['Görsel']).to_excel("trendyol_canli_sonuc.xlsx", index=False)
    # print("Dosya kaydedildi.")
else:
    print("Veri çekilemedi.")

In [ ]:
# 1. GEREKLİ KÜTÜPHANELERİN KURULUMU
!pip install cloudscraper beautifulsoup4 pandas lxml

import cloudscraper
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
from IPython.display import display, HTML

def trendyol_otomasyon_baslat(kategori_linki, taranacak_sayfa_sayisi=3):
    """
    Belirtilen kategori linkindeki ürünleri, sayfa sayfa gezerek toplar.
    Bot korumasını aşmak için CloudScraper ve rastgele bekleme süreleri kullanır.
    """

    # Gerçek bir tarayıcı gibi davranan istekçi (Scraper) oluşturuluyor
    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
    )

    tum_urunler = []
    print(f"🚀 OTOMASYON BAŞLATILIYOR...\n📂 Hedef: {kategori_linki}\n📄 Taranacak Sayfa: {taranacak_sayfa_sayisi}\n")

    base_url = "https://www.trendyol.com"

    for sayfa_no in range(1, taranacak_sayfa_sayisi + 1):
        # ---------------------------------------------------------
        # 1. SAYFA URL'SİNİ OLUŞTURMA (Sonsuz Scroll Simülasyonu)
        # ---------------------------------------------------------
        # Linkte zaten '?' varsa '&pi=' ekle, yoksa '?pi=' ekle
        ayirici = "&" if "?" in kategori_linki else "?"
        hedef_url = f"{kategori_linki}{ayirici}pi={sayfa_no}"

        print(f"🔄 Sayfa {sayfa_no} taranıyor...", end=" ")

        try:
            # ---------------------------------------------------------
            # 2. İSTEK GÖNDERME VE BOT KORUMASI KONTROLÜ
            # ---------------------------------------------------------
            response = scraper.get(hedef_url)

            if response.status_code != 200:
                print(f"❌ HATA: Sayfaya erişilemedi (Kod: {response.status_code})")
                continue

            # ---------------------------------------------------------
            # 3. HTML AYRIŞTIRMA (DOĞRULANMIŞ YAPI)
            # ---------------------------------------------------------
            soup = BeautifulSoup(response.text, 'lxml')

            # Dosya analizinde doğruladığımız sınıf: 'product-card'
            kartlar = soup.select(".product-card")

            # Eğer ana yapı değişmişse yedek sınıfı dene
            if not kartlar:
                kartlar = soup.select(".p-card-wrppr")

            if not kartlar:
                print("⚠️ Bu sayfada ürün bulunamadı (Sayfa sonu olabilir).")
                break

            sayfa_urun_sayisi = 0

            for kart in kartlar:
                try:
                    # MARKA
                    marka_tag = kart.select_one(".product-brand") or kart.select_one(".brand")
                    marka = marka_tag.get_text(strip=True) if marka_tag else "-"

                    # ÜRÜN ADI
                    isim_tag = kart.select_one(".product-name") or kart.select_one(".name")
                    isim = isim_tag.get_text(strip=True) if isim_tag else "-"

                    # FİYAT (Dosyada .price-section içindeydi)
                    fiyat_tag = kart.select_one(".price-section") or kart.select_one(".prc-box-dscntd")
                    fiyat = fiyat_tag.get_text(strip=True) if fiyat_tag else "-"

                    # RESİM
                    img_tag = kart.select_one(".image") or kart.select_one("img")
                    img_url = ""
                    if img_tag:
                        img_url = img_tag.get("src") or img_tag.get("data-src")
                        # Yarım linkleri tamamla (/mnresize/... -> https://cdn...)
                        if img_url and not img_url.startswith("http"):
                            img_url = "https://cdn.dsmcdn.com" + img_url

                    # LİNK
                    urun_linki = kart.get("href")
                    if urun_linki and not urun_linki.startswith("http"):
                        urun_linki = base_url + urun_linki

                    # LİSTEYE EKLE
                    tum_urunler.append({
                        "Görsel": f'<img src="{img_url}" style="width:50px;">',
                        "Marka": marka,
                        "Ürün Adı": isim,
                        "Fiyat": fiyat,
                        "Link": urun_linki,
                        "Sayfa": sayfa_no
                    })
                    sayfa_urun_sayisi += 1

                except Exception as e:
                    continue

            print(f"✅ {sayfa_urun_sayisi} ürün alındı.")

            # ---------------------------------------------------------
            # 4. İNSAN TAKLİDİ (BEKLEME SÜRESİ)
            # ---------------------------------------------------------
            # Trendyol'un bizi robot sanıp engellememesi için 2-4 saniye arası rastgele bekle
            bekleme_suresi = random.uniform(2, 4)
            time.sleep(bekleme_suresi)

        except Exception as hata:
            print(f"❌ Beklenmedik hata: {hata}")
            continue

    return pd.DataFrame(tum_urunler)

# --- AYARLAR VE ÇALIŞTIRMA ---

# 1. Çekmek istediğiniz linki buraya yapıştırın (Örn: Makyaj kategorisi)
HEDEF_LINK = "https://www.trendyol.com/kozmetik-x-c89"

# 2. Kaç sayfa aşağı inilsin?
SAYFA_SAYISI = 3

# 3. Otomasyonu Başlat
df_sonuc = trendyol_otomasyon_baslat(HEDEF_LINK, SAYFA_SAYISI)

# 4. Sonuçları Göster
if not df_sonuc.empty:
    print(f"\n🎉 TEBRİKLER! Toplam {len(df_sonuc)} ürün başarıyla çekildi.\n")

    # Tablo Görünümü Ayarları
    pd.set_option('display.max_colwidth', None)
    display(HTML(df_sonuc.head(10).to_html(escape=False)))

    # Excel Olarak Kaydetmek İçin (İsterseniz başındaki # işaretini kaldırın)
    # df_sonuc.drop(columns=['Görsel']).to_excel("trendyol_otomasyon_sonuc.xlsx", index=False)
    # print("Dosya 'trendyol_otomasyon_sonuc.xlsx' olarak kaydedildi.")
else:
    print("\n⚠️ Hiç ürün çekilemedi. Linki kontrol edin veya IP adresiniz geçici bloklanmış olabilir.")

In [ ]:
import cloudscraper
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
from IPython.display import display, HTML

def trendyol_akilli_takip(kategori_linki):
    """
    Kategorideki ürünleri çeker.
    404 (Sayfa Bulunamadı) hatası alınca otomatik durur ve rapor verir.
    """

    scraper = cloudscraper.create_scraper(
        browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
    )

    tum_urunler = []
    sayfa_no = 1

    print(f"🚀 TARAMA BAŞLATILIYOR...\n📂 Hedef: {kategori_linki}\n")
    print("ℹ️ İşlem bittiğinde veya durdurulduğunda veriler otomatik kaydedilir.\n")

    try:
        while True:
            # 1. URL OLUŞTURMA
            ayirici = "&" if "?" in kategori_linki else "?"
            hedef_url = f"{kategori_linki}{ayirici}pi={sayfa_no}"

            print(f"🔄 Sayfa {sayfa_no} taranıyor...", end=" ")

            try:
                # 2. İSTEK GÖNDERME
                response = scraper.get(hedef_url, timeout=30)

                # --- YENİ EKLENEN KONTROL ---
                # Eğer sayfa yoksa (404), demek ki ürünler bitti.
                if response.status_code == 404:
                    print("🏁 Sayfa bulunamadı (404). Kategori sonuna gelindi.")
                    break # Döngüyü kır ve bitir

                # Diğer hatalar için (Bot koruması vb.) bekleyip tekrar dene
                if response.status_code != 200:
                    print(f"❌ Geçici Hata (Kod: {response.status_code}). 5 sn bekleniyor...")
                    time.sleep(5)
                    continue # Aynı sayfayı tekrar dene

                # 3. HTML ANALİZİ
                soup = BeautifulSoup(response.text, 'lxml')

                kartlar = soup.select(".product-card")
                if not kartlar:
                    kartlar = soup.select(".p-card-wrppr")

                # Kart yoksa da bitir
                if not kartlar:
                    print("🏁 Bu sayfada ürün kartı yok. Tarama tamamlandı.")
                    break

                sayfa_urun_sayisi = 0

                for kart in kartlar:
                    try:
                        marka = kart.select_one(".product-brand") or kart.select_one(".brand")
                        marka = marka.get_text(strip=True) if marka else "-"

                        isim = kart.select_one(".product-name") or kart.select_one(".name")
                        isim = isim.get_text(strip=True) if isim else "-"

                        fiyat = kart.select_one(".price-section") or kart.select_one(".prc-box-dscntd")
                        fiyat = fiyat.get_text(strip=True) if fiyat else "-"

                        img = kart.select_one(".image") or kart.select_one("img")
                        img_url = ""
                        if img:
                            img_url = img.get("src") or img.get("data-src")
                            if img_url and not img_url.startswith("http"):
                                img_url = "https://cdn.dsmcdn.com" + img_url

                        link = kart.get("href")
                        if link and not link.startswith("http"):
                            link = "https://www.trendyol.com" + link

                        tum_urunler.append({
                            "Görsel": f'<img src="{img_url}" style="width:50px;">',
                            "Marka": marka,
                            "Ürün Adı": isim,
                            "Fiyat": fiyat,
                            "Link": link,
                            "Sayfa": sayfa_no
                        })
                        sayfa_urun_sayisi += 1
                    except:
                        continue

                print(f"✅ {sayfa_urun_sayisi} ürün alındı. (Toplam: {len(tum_urunler)})")
                sayfa_no += 1

                # Hızlı gidip engellenmemek için bekleme
                time.sleep(random.uniform(1.5, 3))

            except Exception as e:
                print(f"❌ Bağlantı hatası: {e}")
                time.sleep(5)
                continue

    except KeyboardInterrupt:
        print("\n🛑 Kullanıcı tarafından durduruldu! Çekilen veriler kaydediliyor...")
    except Exception as e:
        print(f"\n❌ Kritik Hata: {e}")

    return pd.DataFrame(tum_urunler)

# --- ÇALIŞTIRMA ---
LINK = "https://www.trendyol.com/kozmetik-x-c89"

# Otomasyonu başlat
df_final = trendyol_akilli_takip(LINK)

# Sonuçları göster ve kaydet
if not df_final.empty:
    print(f"\n🎉 İŞLEM TAMAMLANDI! Toplam {len(df_final)} ürün toplandı.\n")

    # Ekrana bas
    pd.set_option('display.max_colwidth', None)
    display(HTML(df_final.head(5).to_html(escape=False)))

    # Excel olarak kaydet
    dosya_ismi = "trendyol_kozmetik_tum_urunler.xlsx"
    # Resim kodunu (HTML) kaldırarak temiz Excel yapalım
    df_kayit = df_final.drop(columns=['Görsel'])
    df_kayit.to_excel(dosya_ismi, index=False)

    print(f"✅ Veriler '{dosya_ismi}' adıyla Excel dosyasına kaydedildi. Sol menüden indirebilirsiniz.")
else:
    print("Veri yok.")